In [1]:
#Importing the required packages
import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'

import collections
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import librosa
import librosa.display
import IPython.display as ipd
import scipy
import tensorflow as tf
import pandas as pd
import mdn

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Activation, LSTM, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks.callbacks import ModelCheckpoint, CSVLogger
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, minmax_scale, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dwoodw19/miniconda3/envs/ML_ENV

In [ ]:
#The next few boxes are for trying to check if GPUs are availible
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [24]:
#Data stored in "SMC_Dataset1/classical" and "SMC_Dataset1/json"
PATH = os.getcwd()



SpectrogramPath = PATH+'/SMC_dataset1/Spectrograms1024.csv'
MotionPath = PATH+'/SMC_dataset1/motion_features9x5.csv'

#Load Motion features
MotionFeat = pd.read_csv(MotionPath, sep=',')
MotionFeat = np.array(MotionFeat).reshape(1201, 9, 5)

#Load Motion features
Spectro = pd.read_csv(SpectrogramPath, sep=',' ,header=None)



In [25]:
#Feature Scaling/Normalisation of Data 
for i in range(1201):
    MotionFeat[i] = minmax_scale(MotionFeat[i],  feature_range=(0, 1), axis=0, copy=False)
    

#Select 20% of data for validation set
train_x, validate_x, train_y, validate_y = train_test_split(MotionFeat, Spectro, test_size=0.2, random_state=1)


[[4.41337505e+00 1.82696726e+01 2.37263817e-01 1.63819923e+03
  1.92984207e+02]
 [1.85186981e+00 1.08041170e+01 4.29906073e-01 9.57515102e+02
  1.82835435e+02]
 [5.49710570e+00 1.54986648e+01 1.46385572e-01 1.35782101e+03
  6.48934681e+01]
 [5.09279455e+00 9.92993989e+00 3.69880296e-01 7.98039705e+02
  1.39725010e+02]
 [3.91515423e+00 1.33587920e+01 9.96250386e-02 1.10158216e+03
  1.56554556e+02]
 [1.10408430e+01 2.82637538e+01 1.15342907e-01 1.93086098e+03
  7.19054420e+01]
 [5.01365441e+00 3.10260902e+01 1.90316688e-01 1.91160609e+03
  6.07648309e+01]
 [3.24677907e+00 8.23084741e+00 3.13328877e-01 7.84251946e+02
  2.72751723e+02]
 [1.54174314e+00 9.68600089e+00 7.10933205e-01 8.65627047e+02
  7.15255468e+01]]


In [29]:
#Parameters
batch_size=8
N_MIXES = 10  # number of mixture components
OUTPUT_DIMS = 1024  # number of real-values predicted by each mixture component

output shape: 10260


In [31]:
model = Sequential()

#input to model is 5 vectors of 7 features
model.add(LSTM(512, activation = 'sigmoid', return_sequences=True, batch_input_shape=(batch_size, 9, 5)))
model.add(LSTM(1024, activation = 'sigmoid', return_sequences=False))
model.add(Dense(2048, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation = 'relu'))
model.add(Dropout(0.5))
model.add(mdn.MDN(OUTPUT_DIMS, N_MIXES))
model.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMS,N_MIXES), optimizer=RMSprop(learning_rate=0.0001, rho=0.9))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (8, 9, 512)               1060864   
_________________________________________________________________
lstm_5 (LSTM)                (8, 1024)                 6295552   
_________________________________________________________________
dense_6 (Dense)              (8, 2048)                 2099200   
_________________________________________________________________
dropout_6 (Dropout)          (8, 2048)                 0         
_________________________________________________________________
dense_7 (Dense)              (8, 4096)                 8392704   
_________________________________________________________________
dropout_7 (Dropout)          (8, 4096)                 0         
_________________________________________________________________
dense_8 (Dense)              (8, 2048)                

In [32]:
csv_logger = CSVLogger('log.csv', append=True, separator=',')
filepath="MDN/Model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, csv_logger]

history = model.fit(train_x, train_y, batch_size=batch_size, epochs=100, validation_data=(validate_x, validate_y), callbacks=callbacks_list, verbose=0)




Epoch 00001: val_loss improved from inf to 426.02626, saving model to MDN/model30Nmix.hdf5

Epoch 00002: val_loss improved from 426.02626 to 310.51692, saving model to MDN/model30Nmix.hdf5

Epoch 00003: val_loss improved from 310.51692 to 293.64486, saving model to MDN/model30Nmix.hdf5

Epoch 00004: val_loss did not improve from 293.64486

Epoch 00005: val_loss improved from 293.64486 to 201.39999, saving model to MDN/model30Nmix.hdf5

Epoch 00006: val_loss did not improve from 201.39999

Epoch 00007: val_loss improved from 201.39999 to 95.99783, saving model to MDN/model30Nmix.hdf5

Epoch 00008: val_loss did not improve from 95.99783

Epoch 00009: val_loss improved from 95.99783 to 51.26494, saving model to MDN/model30Nmix.hdf5

Epoch 00010: val_loss improved from 51.26494 to 21.18035, saving model to MDN/model30Nmix.hdf5

Epoch 00011: val_loss did not improve from 21.18035

Epoch 00012: val_loss did not improve from 21.18035

Epoch 00013: val_loss improved from 21.18035 to -37.68599

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
